In [2]:
pip install networkx

     |████████████████████████████████| 1.6 MB 3.1 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import networkx as nx

In [36]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [42]:
data = pd.read_csv('full_data2.csv')

/Users/sjk252/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (5,39,49) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [43]:
def network_connections(data):
    """
    This function conducts social network analysis and returns two added features to the data set:
    1) Number of physicians connected to each provider
    2) Number of patients connected to each provider
    """
    Full_soc_dataProv = data.groupby(['Provider','AttendingPhysician'])['PotentialFraud'].count().reset_index()
    Full_soc_dataPatient = data.groupby(['Provider','BeneID'])['PotentialFraud'].count().reset_index()
    
    G1 = nx.Graph()
    G1 = nx.from_pandas_edgelist(Full_soc_dataProv, 'Provider','AttendingPhysician')
    prov_phys_degree = pd.DataFrame(G1.degree)
    prov_phys_degree.columns = ['Provider','AttPhys_Connections']
    prov_phys_degree2 = prov_phys_degree[prov_phys_degree['Provider'].str.contains('PRV')]
    data = pd.merge(prov_phys_degree2, data, how="outer", on="Provider")
    
    G2 = nx.Graph()
    G2 = nx.from_pandas_edgelist(Full_soc_dataPatient, 'Provider','BeneID')
    prov_patient_degree = pd.DataFrame(G2.degree)
    prov_patient_degree.columns = ['Provider','Patient_Connections']
    prov_patient_degree2 = prov_patient_degree[prov_patient_degree['Provider'].str.contains('PRV')]
    data = pd.merge(prov_patient_degree2, data, how="outer", on="Provider")
    
    return data

In [44]:
data = network_connections(data)